In [23]:
import requests
import email
from pathlib import Path
import json

In [ ]:

data = {
    "operationName": "AvailabilityCalendar",
    "variables": {
        "input": {
            "travelPurpose": 2,
            "pagenameDetails": {
                "countryCode": "za",
                "pagename": "brand-new-seapoint-apartment"
            },
            "searchConfig": {
                "searchConfigDate": {
                    "startDate": "2025-05-01",
                    "amountOfDays": 61
                },
                "nbAdults": 2,
                "nbRooms": 4,
                "nbChildren": 0,
                "childrenAges": []
            }
        }
    },
    "extensions": {},
    "query": "query AvailabilityCalendar($input: AvailabilityCalendarQueryInput!) {\n  availabilityCalendar(input: $input) {\n    ... on AvailabilityCalendarQueryResult {\n      hotelId\n      days {\n        available\n        avgPriceFormatted\n        avgPrice\n        checkin\n        minLengthOfStay\n        __typename\n      }\n      __typename\n    }\n    ... on AvailabilityCalendarQueryError {\n      message\n      __typename\n    }\n    __typename\n  }\n}\n"
}
headers = email.message_from_string(headers_str)
r = requests.post("https://www.booking.com/dml/graphql?lang=en", json=data)
r.raise_for_status()
print(json.dumps(r.json(), indent=2))


{
  "data": {
    "availabilityCalendar": {
      "days": [
        {
          "available": true,
          "minLengthOfStay": 2,
          "checkin": "2025-05-30",
          "avgPriceFormatted": "\u00a348",
          "avgPrice": 47.898415924048,
          "__typename": "AvailabilityCalendarDay"
        },
        {
          "__typename": "AvailabilityCalendarDay",
          "avgPrice": 48.7410385654014,
          "minLengthOfStay": 2,
          "checkin": "2025-05-31",
          "avgPriceFormatted": "\u00a349",
          "available": true
        },
        {
          "avgPrice": 49.5836612067549,
          "__typename": "AvailabilityCalendarDay",
          "available": true,
          "checkin": "2025-06-02",
          "minLengthOfStay": 2,
          "avgPriceFormatted": "\u00a350"
        },
        {
          "available": true,
          "avgPriceFormatted": "\u00a350",
          "checkin": "2025-06-01",
          "minLengthOfStay": 2,
          "avgPrice": 49.5836612067549,
  

In [24]:
r = requests.get("https://www.booking.com/hotel/gr/seaside-gouves.html")
r.raise_for_status()    
with Path("response.html").open("w", encoding="utf-8") as f:
    f.write(r.text)

In [27]:
import re 
from collections import defaultdict
from parsel import Selector
def parse_hotel(html: str):
    sel = Selector(text=html)
    css = lambda selector, sep="": sep.join(sel.css(selector).getall()).strip()
    css_first = lambda selector: sel.css(selector).get("")
    # get latitude and longitude of the hotel address:
    result = css_first(".show_map_hp_link::attr(data-atlas-latlng)").split(",")
    print(result)
    # get hotel features by type
    features = defaultdict(list)
    for feat_box in sel.css("[data-capla-component*=FacilitiesBlock]>div>div>div"):
        type_ = feat_box.xpath('.//span[contains(@data-testid, "facility-group-icon")]/../text()').get()
        feats = [f.strip() for f in feat_box.css("li ::text").getall() if f.strip()]
        features[type_] = feats
    data = {
        "title": css("h2#hp_hotel_name::text"),
        "description": css("div#property_description_content ::text", "\n"),
        "address": css(".hp_address_subtitle::text"),
        #"lat": lat,
        #"lng": lng,
        "features": dict(features),
        "id": re.findall(r"b_hotel_id:\s*'(.+?)'", html)[0],
    }
    return data

parse_hotel(r.text)

['']


{'title': '',
 'description': '',
 'address': '',
 'features': {},
 'id': '8727379'}